In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb 
import warnings
warnings.filterwarnings('ignore')

In [29]:
df = pd.read_csv('https://raw.githubusercontent.com/brendenlake/SCAN/master/length_split/tasks_train_length.txt', delimiter=':', header=None)

In [30]:
batch_1 = df[:]

In [31]:
batch_1

,0,1,2
0,IN,walk OUT,I_WALK
1,IN,jump OUT,I_JUMP
2,IN,look OUT,I_LOOK
3,IN,turn right OUT,I_TURN_RIGHT
4,IN,run OUT,I_RUN
...,...,...,...
16985,IN,jump opposite left twice and walk around righ...,I_TURN_LEFT I_TURN_LEFT I_JUMP I_TURN_LEFT I_...
16986,IN,run around right twice and turn opposite left...,I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_...
16987,IN,look left thrice after jump around left twice...,I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_...
16988,IN,look around right twice and walk left thrice OUT,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TUR...


In [32]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [34]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [35]:
np.array(padded).shape

(16990, 3)

In [36]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(16990, 3)

In [37]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [38]:
features = last_hidden_states[0][:,0,:].numpy()

In [39]:
labels = batch_1[1]

In [40]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

In [41]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [42]:
lr_clf.score(test_features, test_labels)

0.0

In [43]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

ValueError: n_splits=5 cannot be greater than the number of members in each class.